In [5]:

import warnings
warnings.filterwarnings("ignore")

In [6]:
from environments.live_environments import BaseLiveTradingEnv
from neuralforecast.core import NeuralForecast
from configs import spot_defaults as defaults
from Keys import *
import pickle
import numpy as np
from utils import pearl_utils
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment
import boto3

In [13]:

conn = boto3.resource('s3')
AWS_ACCESS_KEY
AWS_SECRET_KEY
s3= boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
s3.download_file('coinbasetradehistory','trade.db','Trade_history/trade.db')
# conn.download_file(
#     # Filename='https://coinbasetradehistory.s3.us-east-1.amazonaws.com/trade.db'
# )


In [14]:
forecast_model=NeuralForecast.load(defaults.forecasting_model_path)

Seed set to 6
Seed set to 19
Seed set to 16


In [15]:
base_asset='DOGE'
quote_asset='USDC'
test_net=False
time_frame='1h'
product_type='FUTURE'
product_type='Spot'
futures_target='DOG-29NOV24-CDE'
exchange='coinbase'
trade_target='/'.join([base_asset,quote_asset]) if product_type.upper()=='SPOT' else  futures_target
trade_target

'DOGE/USDC'

In [16]:
live_env=BaseLiveTradingEnv(
            api_key=coinbase_api_key,
            api_secret=coinbase_api_secret,
            paper=test_net,
            symbol=trade_target,
            time_frame=time_frame,
            product_type=product_type,
            positions=[0,1],
            history_path='Trade_history/trade.db',
            exchange=exchange,
            forecast_model=forecast_model,
            discord_webhook='https://discord.com/api/webhooks/986694946381783102/FOA7nG9ShDcXY95-c3XEKV-Fdek66L9xfbQoKuEuFQkK2P4aFWaZ_fKmzw00j8Oj8Woj'

            )

Paper TradingFalse on coinbase at api.coinbase.com
 got symbol DOGE/USDC
final symbol DOGE-USDC
Connected to coinbase client
Getting data


100%|██████████| 78/78 [00:00<00:00, 16397.14it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [17]:
history=live_env.load_history()

In [18]:
history['date']

0                 11-14-2024,
1                 11-14-2024,
2                 11-14-2024,
3                 11-14-2024,
4                 11-14-2024,
5                 11-14-2024,
6                 11-14-2024,
7                 11-14-2024,
8                 11-14-2024,
9                 11-15-2024,
10                11-15-2024,
11          11-15-2024 04:00 
12          11-15-2024 05:00 
13    11-17-2024 10:09 AM PST
14    11-17-2024 10:18 AM PST
15    11-17-2024 10:18 AM PST
16    11-17-2024 10:19 AM PST
17    11-17-2024 10:19 AM PST
18    11-17-2024 10:20 AM PST
19    11-17-2024 10:20 AM PST
20    11-17-2024 04:54 PM PST
21    11-17-2024 04:54 PM PST
Name: date, dtype: object

In [19]:
model_name=defaults.MODEL_NAME
agent_path=f'Agent/pearl_{model_name}_model.pkl'
agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
                                        study_name=f'pearl-{model_name}-hp-search',
                                        action_space_dim=2,
                                            observation_space_dim=30,)
agent=pearl_utils.load_agent_weights(agent,weight_path=agent_path)

AttributeError: module 'configs.spot_defaults' has no attribute 'MODEL_NAME'

In [ ]:
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment

In [ ]:
live_pearl_env=GymEnvironment(live_env)

In [ ]:
observation,action_space=live_pearl_env.reset()

In [ ]:
agent.observe(observation, action_space)

In [ ]:
action=agent.act(exploit=True)

# action=
action

In [ ]:
live_env.client.account()

In [ ]:
action_result=live_pearl_env.step(int(action))


In [ ]:
len(live_env.client.trade_client.get_fills()['fills'])

In [ ]:
action_result.observation

In [ ]:
df=live_env.df
import pandas as pd

In [ ]:
df['ds_pst']=pd.to_datetime(df['ds'],utc=False)
df[['ds','ds_pst']]

In [ ]:
live_env.client.get_trade_rules()